Dans ce notebook, nous avons mis au point un modèle de classification d'images utilisant SVC. 
En exploitant des techniques avancées comme l'extraction de caractéristiques HOG et le rééquilibrage des données avec SMOTE, nous avons optimisé notre modèle avec RandomizedSearchCV pour garantir une performance élevée.

In [3]:
pip install scikit-learn scikit-image pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import os
from skimage import io, color, feature, transform
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE

In [6]:
# Chargement des métadonnées
X = pd.read_csv("C:\\Users\\user\\OneDrive\\DATASIENCETEST\\PROJET\\Data\\Update\\X_train_update.csv", index_col=0)
y = pd.read_csv("C:\\Users\\user\\OneDrive\\DATASIENCETEST\\PROJET\\Data\\Update\\Y_train_CVw08PX.csv", index_col=0).squeeze().map(str)
X['image_path'] = X.apply(lambda row: os.path.join("C:/Users/user/OneDrive/DATASIENCETEST/PROJET/Data/Update/images/image_train", f'image_{row.imageid}_product_{row.productid}.jpg'), axis=1)

In [7]:
# Fonction d'extraction des caractéristiques HOG
def extract_hog_features(image_path):
    image = io.imread(image_path)
    image_gray = color.rgb2gray(image)
    image_resized = transform.resize(image_gray, (128, 64), anti_aliasing=True)
    hog_features = feature.hog(image_resized, pixels_per_cell=(16, 16),
                               cells_per_block=(1, 1), visualize=False)
    return hog_features

# Création du dataset
def create_dataset(X, y):
    features_list = []
    for image_path in X['image_path']:
        features = extract_hog_features(image_path)
        features_list.append(features)
    return np.array(features_list), y.values

In [8]:
# Réduction pour des raisons de démonstration
sample_X = X.head(10000)
sample_y = y.head(10000)

X_features, y_labels = create_dataset(sample_X,sample_y)

In [9]:
# Compter le nombre d'échantillons par classe
unique, counts = np.unique(y_labels, return_counts=True)
counts_dict = dict(zip(unique, counts))

display(counts_dict)

{'10': 385,
 '1140': 299,
 '1160': 462,
 '1180': 84,
 '1280': 557,
 '1281': 241,
 '1300': 598,
 '1301': 93,
 '1302': 298,
 '1320': 392,
 '1560': 609,
 '1920': 522,
 '1940': 87,
 '2060': 612,
 '2220': 94,
 '2280': 549,
 '2403': 566,
 '2462': 157,
 '2522': 578,
 '2582': 303,
 '2583': 1174,
 '2585': 307,
 '2705': 332,
 '2905': 89,
 '40': 293,
 '50': 224,
 '60': 95}

In [10]:
# Rééquilibrage des classes avec SMOTE
smote = SMOTE()
X_res, y_res = smote.fit_resample(X_features, y_labels)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)


In [11]:
# Recalcul des poids de classe après rééquilibrage
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)
class_weights_dict = {class_label: weight for class_label, weight in zip(np.unique(y_train), class_weights)}

display(class_weights_dict)

{'10': 1.0034029756251979,
 '1140': 1.0098765432098766,
 '1160': 1.0077094261643618,
 '1180': 0.9991331757289205,
 '1280': 0.9752701819160802,
 '1281': 1.045863235172812,
 '1300': 1.0186390294850165,
 '1301': 0.9855038669309394,
 '1302': 0.9855038669309394,
 '1320': 0.9917478196253275,
 '1560': 0.9927961788426904,
 '1920': 0.9865390600684718,
 '1940': 0.9938467568097198,
 '2060': 0.9959545972271316,
 '2220': 1.0131447520875785,
 '2280': 0.9917478196253275,
 '2403': 0.9752701819160802,
 '2462': 0.9907016721362712,
 '2522': 1.055264253017062,
 '2582': 1.0077094261643618,
 '2583': 0.9927961788426904,
 '2585': 1.0055515901340313,
 '2705': 0.9875764302683335,
 '2905': 1.0109636008451939,
 '40': 1.0034029756251979,
 '50': 1.0055515901340313,
 '60': 0.977299880525687}

In [13]:
# Optimisation des hyperparamètres avec RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'C': [1, 10],
    'gamma': ['scale'],
    'kernel': ['linear', 'rbf']
}

model = SVC(class_weight=class_weights_dict)

# Configuration pour RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=3, verbose=1, n_jobs=-1, scoring='accuracy')
random_search.fit(X_train, y_train)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomizedSearchCV(cv=3,
                   estimator=SVC(class_weight={'10': 1.0034029756251979,
                                               '1140': 1.0098765432098766,
                                               '1160': 1.0077094261643618,
                                               '1180': 0.9991331757289205,
                                               '1280': 0.9752701819160802,
                                               '1281': 1.045863235172812,
                                               '1300': 1.0186390294850165,
                                               '1301': 0.9855038669309394,
                                               '1302': 0.9855038669309394,
                                               '1320': 0.9917478196253275,
                                               '1560': 0.9927961788426904,
                                               '1920': 0.986539...
                                               '2462': 0.9907016721362712,
                                               '2522': 1.055264253017062,
                                               '2582': 1.0077094261643618,
                                               '2583': 0.9927961788426904,
                                               '2585': 1.0055515901340313,
                                               '2705': 0.9875764302683335,
                                               '2905': 1.0109636008451939,
                                               '40': 1.0034029756251979,
                                               '50': 1.0055515901340313,
                                               '60': 0.977299880525687}),
                   n_jobs=-1,
                   param_distributions={'C': [1, 10], 'gamma': ['scale'],
                                        'kernel': ['linear', 'rbf']},
                   scoring='accuracy', verbose=1)

In [14]:
# Affichage des meilleurs paramètres et de la meilleure score
print("Meilleurs paramètres:", random_search.best_params_)
print("Meilleur score:", random_search.best_score_)

Meilleurs paramètres: {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}
Meilleur score: 0.8749509387831534


In [15]:

# Évaluation sur l'ensemble de test
y_pred = random_search.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9217665615141956
              precision    recall  f1-score   support

          10       0.90      0.92      0.91       238
        1140       0.95      0.98      0.96       244
        1160       0.97      0.96      0.96       242
        1180       0.99      1.00      1.00       234
        1280       0.78      0.83      0.80       211
        1281       0.98      0.93      0.96       276
        1300       0.79      0.85      0.82       252
        1301       1.00      1.00      1.00       221
        1302       0.94      0.96      0.95       221
        1320       0.89      0.92      0.90       227
        1560       0.81      0.79      0.80       228
        1920       0.95      0.93      0.94       222
        1940       0.99      1.00      0.99       229
        2060       0.86      0.81      0.83       231
        2220       0.99      1.00      0.99       247
        2280       0.89      0.87      0.88       227
        2403       0.90      0.84      0.87       211
        

Notre modèle SVC a démontré une capacité exceptionnelle à classer les images de produits, atteignant une précision de plus de 92%. Cette réussite est le fruit d'une préparation minutieuse des données et d'une optimisation rigoureuse des hyperparamètres. Ce travail ouvre la voie à des applications prometteuses pour la classification automatique des produits, marquant une étape importante vers l'automatisation et l'amélioration des processus de catalogage sur Rakuten.